In [1]:
import numpy as np
import pandas as pd



In [2]:
df=pd.read_csv("flights.csv",usecols=["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "ARRIVAL_DELAY"])

len(df)

C:\Users\smala\AppData\Local\Temp\ipykernel_19696\928419362.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("flights.csv",usecols=["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "ARRIVAL_DELAY"])


5819079

In [3]:
df.to_csv("flightsCondensed.csv", index=False)